In [1]:
file_name = "psy_eval_gpt.txt"

In [2]:
api_key = "sk-NYJ30H6jdMFO32ueD6C8116e87Ac435aA9B73a976c87Fc0a"

In [3]:
def constructScoreTable(role_name, sub_factors, score, current_table={}):
    factors_list = sub_factors.split("_")
    unique_list = list(set(factors_list))
    filtered_list = [item for item in unique_list if item != '']
    max_length = 0
    for key, value in current_table.items():
        if len(value) > max_length:
            max_length = len(value)

    unseen_factors = [factor for factor in filtered_list if factor not in current_table.keys()]
    seen_factors = [factor for factor in filtered_list if factor in current_table.keys()]

    for factor in unseen_factors:
        current_table[factor] = max_length * ["null"] + [score]

    for factor in current_table.keys():
        if factor in seen_factors:
            current_table[factor].append(score)
        elif factor in unseen_factors:
            continue
        else:
            current_table[factor].append("null")
    return current_table

In [5]:
prompt = """
You will read a psychological assessment report. This psychological assessment report

assesses whether the subject has a high {dimension} personality.

Based on this report, output a json

containing two fields: score and reason

score is between -5 to 5 points

If the subject shows high {dimension} personality in many factors, the score is 5 points

If the subject shows high {dimension} personality in a single factor, the score is 2 points

If the report is unable to determine the subject's personality, the score is 0 points

If the subject shows low {dimension} personality in a single factor, the score is -2 points

If the subject shows low {dimension} personality in many factors, the score is -5 points

reason is a brief summary of the report

Only output the json, do not output any additional information

Report:
"""

In [6]:
import requests
import json

def get_openai_score(prompt, report):
    prompt = prompt + " \n " + report
    # 设置请求参数
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {api_key}'
    }

    data = {
        "model": "text-davinci-003",
        "prompt": prompt,
        "max_tokens": 1024,
        "temperature": 0.8,
        "n": 5,
        "logprobs": 1
    }

    response = requests.post('https://api.nsxyw.com/v1/completions', headers=headers, json=data)

    if response.status_code == 200:
        result = response.json()
        generated_text = result['choices'][0]['text']
        return generated_text
        # print(generated_text)
    else:
        return generated_text
        # print(response.text)

In [7]:
def extract_subtable(data, prefix):
    subtable = {}
    for key, value in data.items():
        if key.startswith(prefix):
            subtable[key] = value
    return subtable


In [13]:
def get_column_avg(sub_table):
    df = pd.DataFrame(sub_table)
    df = df.replace("null", pd.NA)
    column_means = df.mean()
    column_means = column_means.to_dict()
    return column_means

def get_average_row_means(sub_table):
    df = pd.DataFrame(sub_table)
    df = df.replace('null', pd.NA)
    row_means = df.mean(axis=1)
    average_of_row_means = row_means.mean()
    return average_of_row_means

In [27]:
def indexing_subtable(sub_table):

    global_min_index = float('inf')
    global_max_index = -1

    for key, lst in sub_table.items():
        min_index = float('inf')
        max_index = -1

        for i, val in enumerate(lst):
            if val != "null":
                min_index = min(min_index, i)
                max_index = max(max_index, i)

        global_min_index = min(global_min_index, min_index)
        global_max_index = max(global_max_index, max_index)

    new_table = {}
    for key, lst in sub_table.items():
        lst = lst[global_min_index:global_max_index+1]
        new_table[key] = lst
    print(global_min_index)
    print(global_max_index)
    return new_table


In [30]:
import json

with open(file_name, 'r', encoding='utf-8') as file:
    # 使用readlines()方法按行读取文件内容
    file_lines = file.readlines()

tables_list = []
table = {}
role_name = json.loads(file_lines[0])["role_name"]

for line in file_lines:
    data = json.loads(line)
    score_json = get_openai_score(prompt, data["evaluation"])
    parts = score_json.split("{", 1)  # 第二个参数表示最多拆分成两部分
    if len(parts) > 1:
        result_string = "{" + parts[1]
    else:
        result_string = score_json
    # score_json = json.loads(result_string)
    try:
        score_json = json.loads(result_string)
    except:
        continue

    score = score_json["score"]
    sub_factors = data["sub_factors"]
    if role_name == data["role_name"]:
        table = constructScoreTable(role_name, sub_factors, score, table)
    else:
        N_table = extract_subtable(table, 'N')
        C_table = extract_subtable(table, 'C')
        A_table = extract_subtable(table, 'A')
        O_table = extract_subtable(table, 'O')
        E_table = extract_subtable(table, 'E')

        tables_list.append(table)
        table = constructScoreTable(role_name, sub_factors, score, {})
        with open("result.jsonl", "a") as jsonl_file:
            added_score = {
                "role_name": role_name,
                "N":{
                    "score": get_average_row_means(N_table),
                    "sub_factors": get_column_avg(N_table)
                },
                "C":{
                    "score": get_average_row_means(C_table),
                    "sub_factors": get_column_avg(C_table)
                },
                "A":{
                    "score": get_average_row_means(A_table),
                    "sub_factors": get_column_avg(A_table)
                },
                "O":{
                    "score": get_average_row_means(O_table),
                    "sub_factors": get_column_avg(O_table)
                },
                "E":{
                    "score": get_average_row_means(E_table),
                    "sub_factors": get_column_avg(E_table)
                },
                "N_table" : indexing_subtable(N_table),
                "C_table" : indexing_subtable(C_table),
                "A_table" : indexing_subtable(A_table),
                "O_table" : indexing_subtable(O_table),
                "E_table" : indexing_subtable(E_table)
            }
            json.dump(added_score, jsonl_file, ensure_ascii=False)
            jsonl_file.write('\n')
        role_name = data["role_name"]
    # print(table)


0
4
26
33
18
25
11
17
5
10
0
4
25
32
18
24
11
17
5
10


In [34]:
import pandas as pd

with open("result.jsonl", 'r', encoding='utf-8') as file:
    for line in file:
        # 解析 JSON 数据
        json_data = json.loads(line)

def drawTable(table):
    df = pd.DataFrame(table)
    result = df.to_string(index=False)
    return result

print(drawTable(json_data["E_table"]))
print("\n")
print(drawTable(json_data["N_table"]))

  E2   E6   E4   E1   E3
  -2   -2   -2   -2 null
null    5    5    5    5
null null    2    2    2
   2    2 null null    2
  -2   -2   -2 null null
   2    2    2 null null


  N1  N3  N4   N2   N6
  -2  -2  -2 null null
null  -2  -2   -2   -2
null   2   2 null    2
null  -5  -5 null   -5
null  -2  -2 null   -2
